# Aprendizaje Supervisado de un Conjunto de Datos del Clima

En este notebook se muestra un benchmark de distintos metodos de aprendizaje supervisado, en este caso es un conjunto de datos climatico en Australia.

Este set de datos puede ser descargado del siguiente link:

https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Se iniciara por cargar las bibliotecas necesarias.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn import preprocessing
%matplotlib inline

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

## Preprocesamiento

En esta seccion se realiza el pre-procesamiento sobre el conjunto de datos, este proviente de la investigacion anterior y por lo tanto no es explicara el racionamiento de este pre-procesamiento

In [ ]:
weather_results = pd.read_csv("weatherAUS.csv")
weather_results.replace(to_replace='Yes', value = 1, inplace = True)
weather_results.replace(to_replace='No',  value = 0, inplace = True)

directions = ["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]

def compassToDeg(compass_direction):
    global directions
    index = directions.index(compass_direction)
    angle = index * 22.5
    return angle

def windGustDirConvert(direction):
    global directions
    if direction['WindGustDir'] in directions:
        return compassToDeg(direction['WindGustDir'])

def windDir9amConvert(direction):
    global directions
    if direction['WindDir9am'] in directions:
        return compassToDeg(direction['WindDir9am'])

def windDir3pmConvert(direction):
    global directions
    if direction['WindDir3pm'] in directions:
        return compassToDeg(direction['WindDir3pm'])

windGustDirAngle = weather_results.filter(regex=r'WindGustDir').apply(windGustDirConvert, axis=1)
windDir9amAngle = weather_results.filter(regex=r'WindDir9am').apply(windDir9amConvert, axis=1)
windDir3pmAngle = weather_results.filter(regex=r'WindDir3pm').apply(windDir3pmConvert, axis=1)

windGustDirAngleCos = windGustDirAngle.apply(np.cos)
windGustDirAngleSin = windGustDirAngle.apply(np.sin)
windDir9amAngleCos = windDir9amAngle.apply(np.cos)
windDir9amAngleSin = windDir9amAngle.apply(np.sin)
windDir3pmAngleCos = windDir3pmAngle.apply(np.cos)
windDir3pmAngleSin = windDir3pmAngle.apply(np.sin)

del weather_results['WindGustDir']
del weather_results['WindDir9am']
del weather_results['WindDir3pm']

weather_results['windGustDirAngleCos'] = windGustDirAngleCos
weather_results['windGustDirAngleSin'] = windGustDirAngleSin
weather_results['windDir9amAngleCos'] = windDir9amAngleCos
weather_results['windDir9amAngleSin'] = windDir9amAngleSin
weather_results['windDir3pmAngleCos'] = windDir3pmAngleCos
weather_results['windDir3pmAngleSin'] = windDir3pmAngleSin

weather_results.fillna(weather_results.mean(), inplace = True)

weather_results.sample(10)

La normalizacion se realizara segun el tipo de dato, es decir toda la temperatura se normalizara segun el maximo global de todos los datos de temperatura, igual para la velocidad del viento y otros datos.

In [ ]:
# Normalize all temperature columns
temp_cols = weather_results.filter(regex=r'Temp')
temp_cols = ((temp_cols-temp_cols.min())/(temp_cols.max()-temp_cols.min()))

weather_results['MinTemp'] = temp_cols['MinTemp']
weather_results['MaxTemp'] = temp_cols['MaxTemp']
weather_results['Temp9am'] = temp_cols['Temp9am']
weather_results['Temp3pm'] = temp_cols['Temp3pm']

# Normalize all speed columns
speed_cols = weather_results.filter(regex=r'Speed')
speed_cols = ((speed_cols-speed_cols.min())/(speed_cols.max()-speed_cols.min()))

weather_results['WindGustSpeed'] = speed_cols['WindGustSpeed']
weather_results['WindSpeed9am'] = speed_cols['WindSpeed9am']
weather_results['WindSpeed3pm'] = speed_cols['WindSpeed3pm']


# Normalize all preassure columns
preassure_cols = weather_results.filter(regex=r'Pressure')
preassure_cols = ((preassure_cols-preassure_cols.min())/(preassure_cols.max()-preassure_cols.min()))


weather_results['Pressure9am'] = preassure_cols['Pressure9am']
weather_results['Pressure3pm'] = preassure_cols['Pressure3pm']

# Normalize all humidity columns
humidity_cols = weather_results.filter(regex=r'Humidity')
humidity_cols = ((humidity_cols-humidity_cols.min())/(humidity_cols.max()-humidity_cols.min()))

weather_results['Humidity9am'] = humidity_cols['Humidity9am']
weather_results['Humidity3pm'] = humidity_cols['Humidity3pm']

# Normalize all cloud columns
cloud_cols = weather_results.filter(regex=r'Cloud')
cloud_cols = ((cloud_cols-cloud_cols.min())/(cloud_cols.max()-cloud_cols.min()))

weather_results['Cloud9am'] = cloud_cols['Cloud9am']
weather_results['Cloud3pm'] = cloud_cols['Cloud3pm']

# Normalize remaining columns individually
cols_to_norm = ['Rainfall','Evaporation',
                'Evaporation','Evaporation',
                'Sunshine', 'Cloud9am','Cloud3pm',
                'RainToday','RISK_MM', 'RainTomorrow']
weather_results[cols_to_norm] = weather_results[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Delete non-numerics columns
del weather_results['Date']
del weather_results['Location']

weather_results.sample(10)

## Metodos Supervisados

El set de datos consiste de distintas mediciones de datos meteorologicos ademas de informacion sobre la presencia o no de lluvia en el dia siguiente.

Este problema puede ser modelado como un problema de clasificacion donde lo que se busca es clasificar las distinas mediciones segun la existencia o no de lluvia en el dia siguiente.

Se comenzara por dividir el set de datos en sets de entrenamiento y prueba, estos sets se utilizaran para medir el rendimiento de los distintos metodos.

In [ ]:
msk = np.random.rand(len(weather_results)) < 0.8
train = weather_results[msk]
test = weather_results[~msk]

# Extract the output columns
Y_train = train['RainTomorrow']
Y_test = test['RainTomorrow']

# Extract the output columns
del train['RainTomorrow']
del test['RainTomorrow']

X_train = train
X_test = test

### Linear Regression

El primer metodo en provarse es un metodo de regresion linear. Como lo muestra su nombre este busca aproximaer utilizar por lo que se espera que tenga el rendimiento mas bajo debido a su mayor simplicidad en comparacion de los otros metodos

In [ ]:
logreg = LogisticRegression(solver='liblinear').fit(X_train, Y_train)
print("Training set score: {:.3f}".format(logreg.score(X_train, Y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_test, Y_test)))

### k-Nearest Neigbors

Este metodo es llamado en espanhol los `k` vecinos mas cercanos a una "region" dada en el espacio para determinar a que clase permanece.

El hiperparametro a utilizar en este caso es la cantidad de vecinos `k`, este puede determinarse al obtener multiples cantidades de vecinos para acercarse al valor optimo.

In [ ]:
training_accuracy = []
test_accuracy = []

''' 
# El siguiente bloque de codigo se utilizo para
# obtener la mejor cantidad de vecinos, para el
# notebook solo se ejecuta el caso optimo
neighbors_settings = range(20, 45, 5)
training_accuracy = []
test_accuracy = []
for n_neighbors in neighbors_settings:
    # se construye el modelo de clasificacion
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    clf.fit(X_train, Y_train)
    # se almacena el "training set accuracy"
    training_accuracy.append(clf.score(X_train, Y_train))
    # se almacena la "generalization accuracy"
    test_accuracy.append(clf.score(X_test, Y_test))

plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
'''
clf = KNeighborsClassifier(n_neighbors=25)
clf.fit(X_train, Y_train)
print("Training set score: {:.3f}".format(clf.score(X_train, Y_train)))
print("Test set score: {:.3f}".format(clf.score(X_test, Y_test)))


### Naive Bayes

El naive bayes sigue la formula del teorema de Bayes:

$$P(h|d) = \frac{P(d|h)P(h)}{P(d)}$$

En este caso se busca la probabilidad de que llueva dados los parametros de entrada.

In [ ]:
nbg = GaussianNB().fit(X_train, Y_train)
print("Training set score: {:.3f}".format(nbg.score(X_train, Y_train)))
print("Test set score: {:.3f}".format(nbg.score(X_test, Y_test)))

### Decision Trees

Los arboles de decisiones buscan dividir el espacio segun el segun una serie de "preguntas" segun ciertos atributos.

In [ ]:
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, Y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, Y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, Y_test)))

### Random Forest

El algoritmo de los bosques aleatorios, se encuentra relacionado a los arboles de decisiones. 

Este create una multitud de arboles de deciciones y estima segun el consenso de todos los arboles.

In [ ]:
'''
# El siguiente bloque de codigo se utilizo para
# obtener la mejor cantidad de estimadores, para el
# notebook solo se ejecuta el caso optimo
training_accuracy = []
test_accuracy = []
estimator_settings = range(1, 10)
for n_estimators in estimator_settings:
    # se construye el modelo de clasificacion
    clf = RandomForestClassifier(n_estimators=n_estimators)
    clf.fit(X_train, Y_train)
    # se almacena el "training set accuracy"
    training_accuracy.append(clf.score(X_train, Y_train))
    # se almacena la "generalization accuracy"
    test_accuracy.append(clf.score(X_test, Y_test))

plt.plot(estimator_settings, training_accuracy, label="training accuracy")
plt.plot(estimator_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_estimators")
plt.legend()
'''


rf = RandomForestClassifier(n_estimators=5).fit(X_train, Y_train)
print("Training set score: {:.3f}".format(rf.score(X_train, Y_train)))
print("Test set score: {:.3f}".format(rf.score(X_test, Y_test)))

### Kernel SVM

El nombre de este algoritmo viene de sus siglas en ingles *Support Vector Machine* o maquinas de soporte vectorial en espanhol.

Y busca el vector "optimo" que divide a dos conjuntos de datos. En el caso de este conjunto al tener mas de 2 dimensiones de entrada se tiene un hyperplano para dividir el conjunto de datos.

Por default, como en el codigo que se muestra a continuacion, se utiliza un kernel *RBG* que "anhade" dimensiones extra para obtener resultados no lineales.

**Cuidado**: Este metodo es lento

In [ ]:
svc = SVC(gamma='auto')
svc.fit(X_train, Y_train)
print("Accuracy on training set: {:.3f}".format(svc.score(X_train, Y_train)))
print("Accuracy on test set: {:.3f}".format(svc.score(X_test, Y_test)))

### AdaBoost

AdaBoost busca crear un clasificador "fuerte" basado en una serie de clasificadores mas "debiles". Este utiliza como base *stumps* de decision en lugar de utilizar arboles.

In [ ]:
'''
# El siguiente bloque de codigo se utilizo para
# obtener la mejor cantidad de estimadores, para el
# notebook solo se ejecuta el caso optimo
training_accuracy = []
test_accuracy = []
estimator_settings = range(1, 10)
for n_estimators in estimator_settings:
    # se construye el modelo de clasificacion
    clf = AdaBoostClassifier(n_estimators=n_estimators)
    clf.fit(X_train, Y_train)
    # se almacena el "training set accuracy"
    training_accuracy.append(clf.score(X_train, Y_train))
    # se almacena la "generalization accuracy"
    test_accuracy.append(clf.score(X_test, Y_test))

plt.plot(estimator_settings, training_accuracy, label="training accuracy")
plt.plot(estimator_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_estimators")
plt.legend()
'''

clf = AdaBoostClassifier(n_estimators=1)
clf.fit(X_train, Y_train)
print("Accuracy on training set: {:.3f}".format(clf.score(X_train, Y_train)))
print("Accuracy on test set: {:.3f}".format(clf.score(X_test, Y_test)))

### Voting Classifier

La idea detrás del *Voting Classifier* es combinar clasificadores conceptualmente diferentes de aprendizaje automático y usar un voto mayoritario o las predicciones promedio (voto suave) para predecir las etiquetas.

In [ ]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(n_estimators=10, random_state=1)
clf3 = GaussianNB()

voting = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)])
voting.fit(X_train, Y_train)
print("Accuracy on training set: {:.3f}".format(voting.score(X_train, Y_train)))
print("Accuracy on test set: {:.3f}".format(voting.score(X_test, Y_test)))

## Comparacion de Resultados

En la siguiente tabla se resumen los resultados obtenidos. Se tienen dos resultados uno de entrenamiento y otro prueba para poder identificar sobreajuste de existir.


| Metodo                     | Entrenamiento  | Prueba     |
|----------------------------|----------------|------------|
| Linear Regression          |          0.903 |      0.903 |
| k-Nearest Neighbors (k=25) |          0.829 |      0.816 |
| Naive Bayes                |          0.952 |      0.953 |
| Decision Trees             |          1.000 |      1.000 |
| Random Forest (n=5)       |          1.000 |      1.000 |
| Kernel SVM                 |          0.868 |      0.888 |
| AdaBoost (n=1)            |          1.000 |      1.000 |
| Voting Classifier          |          0.989 |      0.989 |

El metodo con el peor rendimiento es el de kNN, mientras que 3 de los metodos que toman mas de un estimador tienen una precision del 100%.

Ademas el regresor linear, el mas sencillo tiene un muy buen rendimiento. Esto probablemente representa que el conjunto de datos en general tiene un comportamiento muy cercano al lineal, por los metodos son capaces de aproximar esta linearidad con ligeras variaciones como los que tienen mas de un estimador tienen un rendimiento muchos mas alto.

Debido al alto rendimiento de los clasificadores es posible que se puedan remover muchas de las variables del conjunto de datos y no sacrificar el rendimiento.